In [3]:
#Imports
import pandas as pd

#spark
import findspark
findspark.init(r"C:\Users\Lucas\Desktop\ADA\spark-2.3.2-bin-hadoop2.7")

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min
from pyspark.sql.functions import udf
from pyspark.sql.functions import split
from pyspark.sql.functions import explode

from pyspark.sql.types import StringType
from pyspark.sql.types import TimestampType

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()

import warnings
warnings.filterwarnings('ignore')

In [7]:
def get_match_file(leak, charity, node_type):
    return '../generated/inspected_matches/' + node_type +'/' + node_type +'_'+ leak + '_' + charity + '_matches.csv'


In [110]:
def get_matches(leak):
    
    node_types = ['entity']
    charity_types = ['wikipedia', 'INGO', 'forbes']
    
    first = True
    
    matches = None
    
    for node_type in node_types:
        for charity_type in charity_types:
            
            if first:
                matches = spark.read.csv(get_match_file(leak, charity_type, node_type), header=True)
                first = False
            else:
                new_matches = spark.read.csv(get_match_file(leak, charity_type, node_type), header=True)
                matches = matches.union(new_matches)
    
    return matches.drop('_c0')

In [27]:
def filter_edges(edges, nodes):
    '''Given a set of nodes, returns the edges connected to those nodes'''
    entity_ids = nodes.map(lambda r: r[0]).collect()
    return edges.filter(lambda r: r[0] in entity_ids or r[2] in entity_ids)




In [31]:
def filter_nodes(nodes, edges):
    '''Given a set of edges, return the nodes connected to those edges'''
    start_ids = edges.map(lambda r: r[0]).collect()
    end_ids = edges.map(lambda r: r[2]).collect()
    
    return nodes.rdd.filter(lambda r: r[0] in start_ids or r[0] in end_ids)

In [115]:
def get_address_matches(leak):
    edges = spark.read.csv('../data/'+ leak + '/*.edges.csv', header=True)
    
    address_nodes = spark.read.csv('../data/' + leak + '/*address.csv', header=True)
    
    address_edges = edges.rdd.filter(lambda r: r[1] == 'registered_address')
    
    matches=get_matches(leak)
    
    entity_address_edges = filter_edges(address_edges, matches.rdd)
    
    leak_address=filter_nodes(address_nodes,entity_address_edges)
    
    address = spark.createDataFrame(leak_address)
    
    matches_pd=pd.DataFrame(matches.collect(), columns=["node_id","ShellName","CharityName","CharityHeadquarters"])
    
    address_pd=pd.DataFrame(address.collect(), columns=["node_id","name","address","country_codes","countries","sourceID","valid_until","note"])
    
    entity_address_edges_pd=pd.DataFrame(entity_address_edges.collect(),/
                               columns=["node_id","TYPE","END_ID","link","start_date","end_date","sourceID","valid_until"])
    
    matches_pd['node_id']=matches_pd['node_id'].apply(int)
   
    address_pd['node_id']=address_pd['node_id'].apply(int)
    
    address_pd.drop(['note','valid_until','sourceID'],axis=1,inplace=True)

    entity_address_edges_pd['node_id']=entity_address_edges_pd['node_id'].apply(int)
    
    address_pd.rename(columns={'node_id':'END_ID'},inplace=True)
    
    prejudice=matches_pd.merge(entity_address_edges_pd, on ='node_id')
    
    prejudice['END_ID']=prejudice['END_ID'].apply(int)
    
    pride_end=prejudice.merge(address_pd, on = 'END_ID')

    
    return pride_end

In [111]:
    leak='paradise'
    
    edges = spark.read.csv('../data/'+ leak + '/*.edges.csv', header=True)
    
    address_nodes = spark.read.csv('../data/' + leak + '/*address.csv', header=True)
    
    address_edges = edges.rdd.filter(lambda r: r[1] == 'registered_address')
    
    matches=get_matches(leak)
    
    entity_address_edges = filter_edges(address_edges, matches.rdd)
    
    leak_address=filter_nodes(address_nodes,entity_address_edges)
    
    address = spark.createDataFrame(leak_address)
    
    matches_pd=pd.DataFrame(matches.collect(), columns=["node_id","ShellName","CharityName","CharityHeadquarters"])
    
    address_pd=pd.DataFrame(address.collect(), columns=["node_id","name","address","country_codes","countries","sourceID","valid_until","note"])
    
    entity_address_edges_pd=pd.DataFrame(entity_address_edges.collect(), columns=["node_id","TYPE","END_ID","link","start_date","end_date","sourceID","valid_until"])
    
    matches_pd['node_id']=matches_pd['node_id'].apply(int)
   
    address_pd['node_id']=address_pd['node_id'].apply(int)
    
    address_pd.drop(['note','valid_until','sourceID'],axis=1,inplace=True)

    entity_address_edges_pd['node_id']=entity_address_edges_pd['node_id'].apply(int)
    
    address_pd.rename(columns={'node_id':'END_ID'},inplace=True)
    
    prejudice=matches_pd.merge(entity_address_edges_pd, on ='node_id')
    
    prejudice['END_ID']=prejudice['END_ID'].apply(int)
    
    pride_end=prejudice.merge(address_pd, on = 'END_ID')

In [113]:
entity_address_edges.count()

70

In [112]:
matches.count()

106

In [109]:
address_edges.count()

450551

In [116]:
paradise=get_address_matches('paradise')
panama=get_address_matches('panama')
offshore=get_address_matches('offshore')

ValueError: RDD is empty

In [117]:
paradise

,node_id,ShellName,CharityName,CharityHeadquarters,TYPE,END_ID,link,start_date,end_date,sourceID,valid_until,name,address,country_codes,countries
0,82010944,Managed Care,CARE,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
1,82010944,Managed Care,CARE,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
2,82011358,Compass Global Fund,Global Greengrants Fund,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
3,82012727,Vietnam Value Fund,Vietnam Children's Fund,"Unionville, Virginia, United States",registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
4,82011463,International Opportunity IV,Opportunity International,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
5,82019301,ZZZ-FREEDOM DEVELOPMENT TRUST,Children's Development Trust,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
6,82012837,Green World International,World Vision International,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
7,82012837,Green World International,World Vision International,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
8,82010944,Managed Care,CARE,None,registered_address,81010536,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Kirk House 3rd Floor; Albert Panton Street; Ge...,Kirk House 3rd Floor,CYM,Cayman Islands
9,82010944,Managed Care,CARE,None,registered_address,81010536,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Kirk House 3rd Floor; Albert Panton Street; Ge...,Kirk House 3rd Floor,CYM,Cayman Islands


In [119]:
paradise.dropna(subset=['CharityHeadquarters'])

,node_id,ShellName,CharityName,CharityHeadquarters,TYPE,END_ID,link,start_date,end_date,sourceID,valid_until,name,address,country_codes,countries
3,82012727,Vietnam Value Fund,Vietnam Children's Fund,"Unionville, Virginia, United States",registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
12,82012727,Vietnam Value Fund,Vietnam Children's Fund,"Unionville, Virginia, United States",registered_address,81010818,mailing address,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Rua das Faias 978; Quinta da Bicuda; 2750-688 ...,Rua das Faias 978,PRT,Portugal
13,82138671,United Green Foundation,United Nations Foundation,"Washington, D.C., United States",registered_address,81000811,registered office; business address; mailing a...,None,None,Paradise Papers - Appleby,Appleby data is current through 2014,Heiligkreuz 6; Vaduz; 9490; Liechtenstein,Heiligkreuz 6,LIE,Liechtenstein
20,85004797,KLING INTERNATIONAL SERVICES,Population Services International,"Washington, District of Columbia",registered_address,88002083,registered address,None,None,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,BUITENLAND,None,ABW,Aruba
23,85007445,NEW YORK NEW YORK,UJA/Federation of New York,"New York, New York",registered_address,88006824,registered address,None,None,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,"L.G. SMITH BLVD. 148, ORANJESTAD WEST",None,ABW,Aruba
29,85024417,COMMUNITY RESOURCE CENTER,Jewish Community Center,"520 Eighth Avenue, New York, NY 10018 U.S.",registered_address,88013778,registered address,None,None,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,"SEMINOLENSTRAAT 3, SAN NICOLAS NOORD",None,ABW,Aruba
30,85024779,STICHTING SURGYSCOPE FOUNDATION,Stichting INGKA Foundation,Liechtenstein,registered_address,88006236,registered address,None,None,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,"KAMERLINGH ONNESSTRAAT 54-B, ORANJESTAD WEST",None,ABW,Aruba
31,85027973,STICHTING JE FOUNDATION,Stichting INGKA Foundation,Liechtenstein,registered_address,88015505,registered address,None,None,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,"TANKI LEENDERT 151, NOORD",None,ABW,Aruba
35,85031487,STICHTING POSITIVE FOUNDATION,Stichting INGKA Foundation,Liechtenstein,registered_address,88012062,registered address,None,None,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,"SABANA BASORA 154-D, SAVANETA",None,ABW,Aruba
36,85037617,ZION CRECHE CHILD DEVELOPMENT FOUNDATION,Comenius Foundation for Child Development,"Warsaw, Poland",registered_address,88011063,registered address,None,None,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current throu...,"PONTON 90-L, ORANJESTAD WEST",None,ABW,Aruba


In [132]:
    leak='offshore'
    
    edges = spark.read.csv('../data/'+ leak + '/*.edges.csv', header=True)
    
    address_nodes = spark.read.csv('../data/' + leak + '/*address.csv', header=True)
    
    address_edges = edges.rdd.filter(lambda r: r[1] == 'registered_address')
    
    matches=get_matches(leak)
    
    entity_address_edges = filter_edges(address_edges, matches.rdd)
    
    leak_address=filter_nodes(address_nodes,entity_address_edges)
    
    address = spark.createDataFrame(leak_address)

ValueError: Some of types cannot be determined by the first 100 rows, please try again with sampling

In [134]:
address_nodes.take(10)

[Row(node_id='2004267', name=None, address='One Bearer Secured Debenture', country_codes='XXX', countries='Not identified', sourceID='Offshore Leaks', valid_until='The Offshore Leaks data is current through 2010', note=None),
 Row(node_id='67277', name=None, address='11 Coomber Road, The Peak, Hong Kong', country_codes='HKG', countries='Hong Kong', sourceID='Offshore Leaks', valid_until='The Offshore Leaks data is current through 2010', note=None),
 Row(node_id='67346', name=None, address='4 Irish Place 2nd Floor, Gibraltar.', country_codes='GIB', countries='Gibraltar', sourceID='Offshore Leaks', valid_until='The Offshore Leaks data is current through 2010', note=None),
 Row(node_id='67278', name=None, address='11A, Branksome 3 Tregunter Path Mid-Levels Hong Kong', country_codes='HKG', countries='Hong Kong', sourceID='Offshore Leaks', valid_until='The Offshore Leaks data is current through 2010', note=None),
 Row(node_id='67282', name=None, address='14 Lysander Road West Mailing Kent M